In [ ]:
!pip install mediapipe==0.9.3 protobuf==3.20.*
!pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 10.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import mediapipe as mp
import zipfile
import py7zr
from google.colab import drive
from tensorflow.keras.utils import to_categorical
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/KArSL-502'
output_base_path='/content/kArSL-502'
if not os.path.exists(output_base_path):
          os.makedirs(output_base_path)
num_classes = 502

Mounted at /content/drive


In [ ]:
df=pd.read_excel(os.path.join(base_path,"KARSL-502_Labels.xlsx"))
df.head()

,SignID,Sign-Arabic,Sign-English
0,1,0,0
1,2,1,1
2,3,2,2
3,4,3,3
4,5,4,4


In [ ]:
id_to_label={}
for index,row in df.iterrows():
  id=row['SignID']
  label=row['Sign-English']
  id_to_label[id]=label

In [ ]:
df_trains=[]
df_tests=[]

for item in os.listdir(base_path):
  if (os.path.isdir(os.path.join(base_path,item))):
    for subitem in os.listdir(os.path.join(base_path,item)):
      for zipfile in os.listdir(os.path.join(base_path,item,subitem)):
        #if zipfile.startswith('000'):
          output_path = os.path.join(output_base_path, item, subitem, zipfile.rstrip('.7z'))
          if not os.path.exists(output_path):
            os.makedirs(output_path)
          with py7zr.SevenZipFile(os.path.join(base_path,item,subitem,zipfile), 'r') as archive:
            archive.extractall(output_path)

In [ ]:
# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=True, model_complexity=1, smooth_landmarks=True)

# Initialize MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(color=(128,0,128),thickness=1, circle_radius=1)
connect_drawing_spec = mp_drawing.DrawingSpec(color=(200,200,200),thickness=1, circle_radius=1)

def mediapipe_detection(image, model):
  results = holistic.process(image)
  image_copy=image.copy()
  if results.pose_landmarks:
      mp_drawing.draw_landmarks(
          image_copy, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
          landmark_drawing_spec=drawing_spec, connection_drawing_spec=drawing_spec)
  if results.face_landmarks:
      mp_drawing.draw_landmarks(
          image_copy, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
          landmark_drawing_spec=drawing_spec, connection_drawing_spec=connect_drawing_spec)
  if results.left_hand_landmarks:
      mp_drawing.draw_landmarks(
          image_copy, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
          landmark_drawing_spec=drawing_spec, connection_drawing_spec=drawing_spec)
  if results.right_hand_landmarks:
      mp_drawing.draw_landmarks(
          image_copy, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
          landmark_drawing_spec=drawing_spec, connection_drawing_spec=drawing_spec)
  return image_copy, results

In [ ]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [ ]:
def pad_or_truncate_videos(videos, max_len=25, frame_size=126):
    padded_videos = []
    for video in videos:
        num_frames = video.shape[0]
        if num_frames < max_len:
            # Pad with zeros
            padding = np.zeros((max_len - num_frames, frame_size))
            padded_video = np.vstack((video, padding))
        else:
            # Truncate to max_len
            padded_video = video[:max_len]
        padded_videos.append(padded_video)
    return np.array(padded_videos)

In [ ]:
def process_batch(videos):

  videos_landmarks=[]

  # Process each image
  for video in videos:
    images_landmarks = []
    for image in video:
        holistic_image, results = mediapipe_detection(image, holistic)
        landmarks=extract_keypoints(results)
        images_landmarks.append(landmarks)
    images_landmarks = np.array(images_landmarks)
    videos_landmarks.append(images_landmarks)
  padded_landmarks=pad_or_truncate_videos(videos_landmarks)
  print("made  it")
  return padded_landmarks

In [ ]:
def load_videos_in_batches(folder_paths, batch_size):
    all_videos = []
    all_labels = []
    current_batch_size = 0

    for folder_path, label in folder_paths:
        images = []
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            image = mpimg.imread(image_path)
            images.append(image)

        if images:
            all_videos.append(np.array(images))
            all_labels.append(label)
            current_batch_size += 1

        if current_batch_size >= batch_size:
            yield all_videos, np.array(all_labels)
            all_videos = []
            all_labels = []
            current_batch_size = 0

    if all_videos:
        yield all_videos, np.array(all_labels)

In [ ]:
def encode_labels(y):
  y_train = to_categorical(y - 1, num_classes=num_classes)
  return y_train

In [ ]:
def train_model(X_train,batch_labels):
  X_train, X_val, y_train, y_val = train_test_split(X_train, batch_labels, test_size=0.2, stratify=batch_labels, random_state=42)

  # Convert the labels back to one-hot encoding after splitting
  y_train = encode_labels(y_train)
  y_val = encode_labels(y_val)
  # Train the model with validation data
  print("training")
  history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val),
                      callbacks=[early_stopping])



In [ ]:
def process_data_in_batches(output_base_path, batch_size):
    folder_paths = []

    for fold in os.listdir(output_base_path):
        for subfold in os.listdir(os.path.join(output_base_path, fold)):
            if subfold == "train":
                signer_folder_path = os.path.join(output_base_path, fold, subfold)
                for i, folder in enumerate(os.listdir(signer_folder_path)):
                    folder_path = os.path.join(signer_folder_path, folder)
                    for j, idfolder in enumerate(os.listdir(folder_path)):
                        idfolder_path = os.path.join(folder_path, idfolder)
                        for k, subfolder in enumerate(os.listdir(idfolder_path)):
                            subfolder_path = os.path.join(idfolder_path, subfolder)
                            folder_paths.append((subfolder_path, int(idfolder)))

    for batch_videos, batch_labels in load_videos_in_batches(folder_paths, batch_size):
        print(len(batch_videos),batch_videos[0].shape)
        videos_landmarks=process_batch(batch_videos)
        print(videos_landmarks.shape)
        train_model(videos_landmarks,batch_labels)

In [ ]:
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(25, 42 * 3)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
batch_size = 100

process_data_in_batches(output_base_path, batch_size)

100 (40, 256, 256, 3)
made  it
(100, 25, 126)
training
Epoch 1/50
3/3 [==============================] - 8s 513ms/step - loss: 6.2111 - accuracy: 0.0000e+00 - val_loss: 6.1596 - val_accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================] - 0s 120ms/step - loss: 6.0723 - accuracy: 0.0000e+00 - val_loss: 5.4855 - val_accuracy: 0.0000e+00
Epoch 3/50
3/3 [==============================] - 0s 120ms/step - loss: 5.8538 - accuracy: 0.0000e+00 - val_loss: 5.0874 - val_accuracy: 0.0000e+00
Epoch 4/50
3/3 [==============================] - 0s 101ms/step - loss: 5.0373 - accuracy: 0.1875 - val_loss: 4.3924 - val_accuracy: 0.4000
Epoch 5/50
3/3 [==============================] - 0s 99ms/step - loss: 4.1128 - accuracy: 0.4250 - val_loss: 3.3771 - val_accuracy: 0.4000
Epoch 6/50
3/3 [==============================] - 0s 121ms/step - loss: 3.1173 - accuracy: 0.4250 - val_loss: 3.0452 - val_accuracy: 0.4000
Epoch 7/50
3/3 [==============================] - 0s 130ms/step - loss: 2.7154 - a

In [ ]:
model.save('Sign-Translator.h5')